In [90]:
import pandas as pd
import numpy as np

In [91]:
arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'],
          ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
##or is the same as
arrays = [np.array(['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux']),
          np.array(['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two'])]

In [92]:
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
s = pd.Series(np.random.randn(8), index=index)

In [93]:
iterables = [['bar', 'baz', 'foo', 'qux'], ['one', 'two']]
pd.MultiIndex.from_product(iterables, names=['first', 'second'])

MultiIndex(levels=[['bar', 'baz', 'foo', 'qux'], ['one', 'two']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['first', 'second'])

In [94]:
index.get_level_values(0)
index.get_level_values('second')
s.reindex(index[:3])

first  second
bar    one       0.342430
       two      -1.321121
baz    one       0.661244
dtype: float64

In [95]:
s.reindex([('foo', 'two'), ('bar', 'one'), ('qux', 'one'), ('baz', 'one')])

first  second
foo    two      -0.610155
bar    one       0.342430
qux    one       0.974840
baz    one       0.661244
dtype: float64

In [96]:
s.loc[('bar', 'two'),] ##tuple

-1.3211207410946446

In [97]:
s.loc[('baz', 'two'):('qux', 'one')]

first  second
baz    two      -0.503453
foo    one       0.177628
       two      -0.610155
qux    one       0.974840
dtype: float64

In [98]:
sd = pd.Series([1, 2, 3, 4, 5, 6],index=pd.MultiIndex.from_product([["A", "B"], ["c", "d", "e"]]))
sd.loc[(['A','B']),(['c','d'])]

A  c    1
   d    2
B  c    4
   d    5
dtype: int64

In [99]:
def mklbl(prefix,n):
    return ["%s%s" % (prefix,i)  for i in range(n)]
miindex = pd.MultiIndex.from_product(
    [mklbl('A',4),
     mklbl('B',2),
     mklbl('C',4),
     mklbl('D',2)])
micolumns = pd.MultiIndex.from_tuples(
    [('a','foo'),('a','bar'),('b','foo'),('b','bah')], names=['lvl0', 'lvl1'])
dfmi = pd.DataFrame(
    np.arange(len(miindex)*len(micolumns)).reshape((len(miindex),len(micolumns))),
    index=miindex, columns=micolumns).sort_index().sort_index(axis=1)

In [100]:
dfmi.loc[(slice('A1','A3'), slice(None), ['C1', 'C3']), :].head()
## with 4 slicing element with 'loc'
## huge data could be classified in huge index and small values
## 

lvl0           a         b     
lvl1         bar  foo  bah  foo
A1 B0 C1 D0   73   72   75   74
         D1   77   76   79   78
      C3 D0   89   88   91   90
         D1   93   92   95   94
   B1 C1 D0  105  104  107  106

In [101]:
## slicing multi index and column date with two arguments with 'IndexSlice'
idx = pd.IndexSlice
dfmi.loc[idx[:, :, ['C1', 'C3']], idx[:, 'foo']]
##or use tuple () 
dfmi.loc['A1', (slice(None), 'foo')].head()

lvl0       a   b
lvl1     foo foo
B0 C0 D0  64  66
      D1  68  70
   C1 D0  72  74
      D1  76  78
   C2 D0  80  82

In [102]:
mask = dfmi[('a', 'foo')] >= 200
dfmi.loc[idx[mask,:,['C1','C3']],idx[:,'foo']]

lvl0           a    b
lvl1         foo  foo
A3 B0 C1 D0  200  202
         D1  204  206
      C3 D0  216  218
         D1  220  222
   B1 C1 D0  232  234
         D1  236  238
      C3 D0  248  250
         D1  252  254

In [103]:
dfmi.loc(axis=0)[:, :, ['C1']].head()

lvl0          a       b    
lvl1        bar foo bah foo
A0 B0 C1 D0   9   8  11  10
         D1  13  12  15  14
   B1 C1 D0  41  40  43  42
         D1  45  44  47  46
A1 B0 C1 D0  73  72  75  74

In [104]:
## If you wanna access to single index of multiIndex use 'xs'

In [105]:
dfmi.xs('B0', level=1).loc[idx['A0',:,:],:].iloc[:,0] 
## get first B0 only, A0 only, and first column only

A0  C0  D0     1
        D1     5
    C1  D0     9
        D1    13
    C2  D0    17
        D1    21
    C3  D0    25
        D1    29
Name: (a, bar), dtype: int64

In [106]:
## xs('B0', level=1, drop_level=False) 
dfmi.loc[idx['A0','B0',:,],('a','bar')] 

A0  B0  C0  D0     1
            D1     5
        C1  D0     9
            D1    13
        C2  D0    17
            D1    21
        C3  D0    25
            D1    29
Name: (a, bar), dtype: int64

In [107]:
## set an axis for xs
dfmi.xs('a', level=0, axis=1).loc[('A0','B0')] 

lvl1   bar  foo
C0 D0    1    0
   D1    5    4
C1 D0    9    8
   D1   13   12
C2 D0   17   16
   D1   21   20
C3 D0   25   24
   D1   29   28

In [108]:
dfmi.xs(('bar', 'a'), level=(1,0), axis=1).loc[('A0','B0')] 

lvl0    a
lvl1  bar
C0 D0   1
   D1   5
C1 D0   9
   D1  13
C2 D0  17
   D1  21
C3 D0  25
   D1  29

In [109]:
midx = pd.MultiIndex(levels=[['zero', 'one'], ['x','y']],labels=[[1,1,0,0],[1,0,1,0]])
df = pd.DataFrame(np.random.randn(4,2), index=midx)

In [110]:
##Alignment
df2 = df.mean(level=0)
df2.reindex(df.index, level=0)

0         1
one  y -0.823088  0.279263
     x -0.823088  0.279263
zero y -0.102698  0.101149
     x -0.102698  0.101149

In [111]:
df_aligned, df2_aligned = df.align(df2, level=0)
df.swaplevel(0,1,axis=0)

,,0,1
y,one,-2.275687,0.486186
x,one,0.629511,0.072341
y,zero,0.082992,0.276551
x,zero,-0.288389,-0.074254


In [112]:
All = slice(None); swa = dfmi.loc[['A0'],All]
swa.swaplevel(0,3, axis=0).head()

lvl0          a       b    
lvl1        bar foo bah foo
D0 B0 C0 A0   1   0   3   2
D1 B0 C0 A0   5   4   7   6
D0 B0 C1 A0   9   8  11  10
D1 B0 C1 A0  13  12  15  14
D0 B0 C2 A0  17  16  19  18

In [113]:
#multiple swapping
temp = df.reorder_levels([1,0],axis=0)
swa.reorder_levels([3,2,1,0], axis=0).sort_index().xs('D0',level=0, drop_level=False)

lvl0          a       b    
lvl1        bar foo bah foo
D0 C0 B0 A0   1   0   3   2
      B1 A0  33  32  35  34
   C1 B0 A0   9   8  11  10
      B1 A0  41  40  43  42
   C2 B0 A0  17  16  19  18
      B1 A0  49  48  51  50
   C3 B0 A0  25  24  27  26
      B1 A0  57  56  59  58

In [114]:
dfm = pd.DataFrame({'jim': [0, 0, 1, 1],
                    'joe': ['x', 'x', 'z', 'y'],
                    'jolie': np.random.rand(4)})
dfm = dfm.set_index(['jim', 'joe']); dfm ##sort depth is 1

jolie
jim joe          
0   x    0.001333
    x    0.925118
1   z    0.938215
    y    0.266228

In [115]:
dfm.index.is_lexsorted() ##False, sorted only 1 depth out of 2 indices
dfm = dfm.sort_index()

In [116]:
dfm.index.lexsort_depth ##sort depth is 2

2

In [117]:
## 'axes' is also the plural of ax and axe, but it is also the plural of axis

In [118]:
df = pd.DataFrame({'A': [1, 2, 3, 4]}, index=pd.IntervalIndex.from_breaks([0, 1, 2, 3, 4]))

In [119]:
c = pd.cut(range(4), bins=2)

In [120]:
c.categories

IntervalIndex([(-0.003, 1.5], (1.5, 3.0]]
              closed='right',
              dtype='interval[float64]')

In [121]:
pd.cut([0, 3, 5, 1], bins=c.categories)

[(-0.003, 1.5], (1.5, 3.0], NaN, (-0.003, 1.5]]
Categories (2, interval[float64]): [(-0.003, 1.5] < (1.5, 3.0]]

In [122]:
df = pd.DataFrame(index=[2,3,3,4,5], columns=['data'], data=list(range(5)))
df.index.is_monotonic_increasing

True

In [123]:
weakly_monotonic = pd.Index(['a', 'b', 'c', 'c'])
weakly_monotonic.is_monotonic_increasing

True

In [124]:
weakly_monotonic.is_monotonic_increasing & weakly_monotonic.is_unique

False